In [2]:
#Using Microsoft Cognitive Toolkit CNTK
import cntk
cntk.__version__

'2.5.1'

In [6]:
cntk.minus([1,2,3], [4,5,6]) #It will give the type of operation taht will be performed and the dimension of the array constants
cntk.minus([1,2,3], [4,5,6]).eval() #It will generate the actual output

array([-3., -3., -3.], dtype=float32)

In [12]:
#A slightly more interesting example that uses input variables 
import numpy as np
x=cntk.input_variable(2)
y=cntk.input_variable(2)
x0=np.asarray([[2.,1.]], dtype=np.float32)
y0=np.asarray([[4.,6.]], dtype=np.float32)
cntk.squared_error(x,y).eval({x:x0,y:y0}) #Operation= ((2-4)^2 +(1-6)^2) = 4+25=29


array([29.], dtype=float32)

In [13]:
import cntk as C
c=C.constant(3, shape=(3,3))
print(c.asarray())
np.ones_like(c.asarray())

[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [14]:
#Setup a network that will learn a classifier with fully connected layers using only the 
#functions Sequential() and Dense() from the Layers Library

from __future__ import print_function
import numpy as np
import cntk as C
from cntk.learners import sgd
from cntk.logging import ProgressPrinter
from cntk.layers import Dense, Sequential

In [15]:
def generate_random_data(sample_size, feature_dim, num_classes):
    
    #Create synthetic data using numpy
    Y=np.random.randint(size=(sample_size, 1), low=0, high=num_classes)
    #Make sure the data is separable
    X=(np.random.randn(sample_size, feature_dim)+3)*(Y+1)
    X=X.astype(np.float32)
    #Converting class 0 into vector "1 0 0", 
    #class 1 into vector "0 1 0", ...
    class_ind= [Y == class_number for class_number in range(num_classes)]
    Y=np.asarray(np.hstack(class_ind), dtype=np.float32)
    return X, Y

In [20]:
def ffnet():
    inputs=2
    outputs=2
    layers=2
    hidden_dimension=50
    
    #Input variables denoting the features and label data
    features=C.input_variable((inputs), np.float32)
    label=C.input_variable((outputs), np.float32)
    
    #Instantiate the Classification Feed Forward model
    my_model=Sequential([
        Dense(hidden_dimension, activation=C.sigmoid),
        Dense(outputs)])
    z=my_model(features)
    
    ce=C.cross_entropy_with_softmax(z, label)
    pe=C.classification_error(z, label)
    
    #Instantiate the trainer object to drive the model training
    lr_per_minibatch=C.learning_parameter_schedule(0.125)
    progress_printer=ProgressPrinter(0)
    trainer=C.Trainer(z, (ce,pe), [sgd(z.parameters, lr=lr_per_minibatch)], [progress_printer])
    
    #Get mini-batches of training data and perform model training
    minibatch_size=25
    num_minibatches_to_train=1024
    
    aggregate_loss=0.0
    for i in range(num_minibatches_to_train):
        train_features, labels = generate_random_data(minibatch_size, inputs, outputs)
        #Specify the mapping of input variables in the mdoel to actual minibatch data to be trained with
        trainer.train_minibatch({features:  train_features, label: labels})
        sample_count=trainer.previous_minibatch_sample_count
        aggregate_loss+=trainer.previous_minibatch_loss_average*sample_count
        
    last_avg_error=aggregate_loss/trainer.total_number_of_samples_seen
    
    test_features, test_labels=generate_random_data(minibatch_size, inputs, outputs)
    avg_error=trainer.test_minibatch({features: test_features, label: test_labels})
    print(' error rate of an unseen minibatch: {}'.format(avg_error))
    return last_avg_error, avg_error

np.random.seed(12345)
ffnet()        
        
    
    

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.125
     0.78       0.78        0.6        0.6            25
    0.716      0.684       0.52       0.48            75
    0.685      0.661      0.491       0.47           175
    0.617      0.558      0.309       0.15           375
    0.548      0.484      0.223      0.142           775
    0.465      0.385      0.156       0.09          1575
    0.384      0.303      0.122     0.0887          3175
    0.324      0.264      0.105     0.0878          6375
    0.278      0.233     0.0932     0.0817         12775
    0.247      0.216     0.0863     0.0794         25575
 error rate of an unseen minibatch: 0.08


(0.2469795736670494, 0.08)